# Import dependencies

In [2]:
import os
import sys
import numpy as np
import scipy as sp
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab


import emcee
from emcee import PTSampler
from PyAstronomy.pyasl import MarkleyKESolver
from decimal import Decimal  # histograms
import corner
import time as chrono
import multiprocessing
import datetime as dt

import emperors_library as emplib
import emperors_mirror as empmir

try:
    from tqdm import tqdm
except ImportError:
    raise ImportError('You don t have the package tqdm installed. Try pip install tqdm.')
try:
    from termcolor import colored
except:
    print('You are missing the most cool package in Python! Try pip install termcolor')
try:
    from pygame import mixer
except:
    print('You are missing the most cool package in Python! Try pip install pygame or set MUSIC=False')


You are missing the most cool package in Python! Try pip install pygame or set MUSIC=False


# Model Functions

In [ ]:
def RV_model(THETA, time, kplanets):
    modelo = 0.0
    if kplanets == 0:
        return 0.0
    for i in range(kplanets):
        P, As, Ac, S, C = THETA[5*i:5*(i+1)]
        A = As ** 2 + Ac ** 2
        ecc = S ** 2 + C ** 2
        w = sp.arccos(C / (ecc ** 0.5))  # longitude of periastron
        phase = sp.arccos(Ac / (A ** 0.5))
        ### test
        if S < 0:
            w = 2 * sp.pi - sp.arccos(C / (ecc ** 0.5))
        if As < 0:
            phase = 2 * sp.pi - sp.arccos(Ac / (A ** 0.5))

        ###
        per = sp.exp(P)
        freq = 2. * sp.pi / per
        M = freq * time + phase  # mean anomaly
        E = sp.array([MarkleyKESolver().getE(m, ecc) for m in M])  # eccentric anomaly
        f = (sp.arctan(((1. + ecc) ** 0.5 / (1. - ecc) ** 0.5) * sp.tan(E / 2.)) * 2.)  # true anomaly
        modelo += A * (sp.cos(f + w) + ecc * sp.cos(w))
    return  modelo


def transit_lightCurve(lc_params, time, free_params, ld, *theta_j):
    flux = 0.0
    params = batman.TransitParams()
    nfp = len(free_params)
    f_params = (nfp + ld) * fsig
    if theta_j:
        params.per = theta_j[3*i]
        params.w = theta_j[3*i + 1]
        params.ecc = theta_j[3*i + 2]
    else:
        params.per = lc_params[(nfp+ld)*(i+1) - 2]
        params.w = lc_params[(nfp+ld)*(i+1) - 1]
        params.ecc = lc_params[(nfp+ld)*(i+1)]
    #params.per = P                       #orbital period in days
    params.t0 = lc_params[(nfp+ld)*i]                       #time of inferior conjunction
    params.rp = lc_params[(nfp+ld*i) + 1]                   #planet radius (in units of stellar radii)
    params.a = lc_params[(nfp+ld*i) + 2]                      #semi-major axis (in units of stellar radii)
    params.inc = lc_params[(nfp+ld*i) + 3]                     #orbital inclination (in degrees)
    #params.ecc = 0.                      #eccentricity

    #params.w = 0.                        #longitude of periastron (in degrees)
    u = []
    for j in range(ld):
        u.append(lc_params[nfp+j + (nfp+ld)*i])
    params.u = u                #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"       #limb darkening model
    m = batman.TransitModel(params, time)    #initializes model
    flux += m.light_curve(params)          #calculates light curve
    #print flux, 'flux2\n\n'  # PMPMPM
    return (flux)


# Dummy functions

In [3]:
def logp(theta, func_logp, args):
    return func_logp(theta, args)


def logl(theta, func_logl, args):
    return func_logl(theta, args)

In [3]:
def uniform(x, lims, *args):
    if lims[0] <= x <= lims[1]:
        return 1.0
    
def flat(x, lims, *args):
    return 0.0

In [4]:
d = {'uniform':uniform,
     'flat':flat}

In [5]:
import scipy as sp
d['uniform']

<function __main__.uniform(x, lims, *args)>

In [6]:
class spec:
    def __init__(self, name, units, prior, lims, val, *args):
        self.name = name
        self.units = units
        self.prior = d[str(prior)]
        self.lims = lims
        self.val = -sp.inf
    def p(self, x, *args):
        return self.prior(x, args)
    def identify(self):
        return self.name+'    '+self.units
    def change(self, thing, whatto):
        pass
        
        

In [7]:
theta1 = spec('Acceleration', '[m/s]', 'uniform', [0, 1], 1)

In [8]:
theta1.name

'Acceleration'

In [13]:
t = sp.array([])
t = sp.append(t, spec('Acceleration', '[m/s]', 'uniform', [0, 1], 1))
t = sp.append(t, spec('Acceleration1', '[m/s]', 'uniform', [0, 1], 1))
t = sp.append(t, spec('Acceleration2', '[m/s]', 'uniform', [0, 1], 1))

In [14]:
t

array([<__main__.spec object at 0x7f0674229438>,
       <__main__.spec object at 0x7f0674229550>], dtype=object)

In [16]:
t[1].name

'Acceleration1'

In [18]:
t[1].xd = 1

In [19]:
t[1].xd

1

In [28]:
def change_name(object_id, action, whato):
    for theta in t:
        if theta.name == object_id:
            setattr(theta, action, whato)
    pass

In [30]:
change_name('Acceleration1', 'name', 'works!!')

In [35]:
t[1].name

'works!!'

In [ ]:
class obje